In [164]:
import re
import os
import datetime
import gzip
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
import gffutils

if os.getcwd().endswith('notebook'):
    os.chdir('..')

In [3]:
sns.set(palette='colorblind', font_scale=1.3)

## Load traits

In [8]:
species_traits_path = os.path.join(os.getcwd(), 'data/condensed_traits/condensed_species_NCBI_with_ogt.csv')
species_traits_all = pd.read_csv(species_traits_path)
len(species_traits_all)

11265

In [10]:
species_traits_all['species_taxid'] = species_traits_all['species_tax_id']

In [11]:
species_traits_all.head()

,species_tax_id,species,genus,family,order,class,phylum,superkingdom,gram_stain,metabolism,...,gc_content.stdev,coding_genes.stdev,optimum_tmp.stdev,optimum_ph.stdev,growth_tmp.stdev,rRNA16S_genes.stdev,tRNA_genes.stdev,data_source,ref_id,species_taxid
0,1416943,Lactobacillus sicerae,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria,NaN,anaerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,campedelli,152,1416943
1,1229250,Ammoniibacillus agariperforans,Ammoniibacillus,Paenibacillaceae,Bacillales,Bacilli,Firmicutes,Bacteria,NaN,aerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,181,1229250
2,1081799,Thermomarinilinea lacunifontana,Thermomarinilinea,Anaerolineaceae,Anaerolineales,Anaerolineae,Chloroflexi,Bacteria,NaN,aerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,206,1081799
3,1323370,Caloranaerobacter ferrireducens,Caloranaerobacter,Clostridiaceae,Clostridiales,Clostridia,Firmicutes,Bacteria,NaN,anaerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,380,1323370
4,192731,Halomicronema excentricum,Halomicronema,Leptolyngbyaceae,Synechococcales,NaN,Cyanobacteria,Bacteria,NaN,aerobic,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,corkrey,406,192731


## Load NCBI species short-list

In [334]:
ncbi_species_metadata = pd.read_csv(
    os.path.join(os.getcwd(), 'data/condensed_traits/ncbi_species_final.csv'),
    parse_dates=['seq_rel_date_processed'],
)
len(ncbi_species_metadata)

2359

In [335]:
ncbi_species_metadata.head()

,assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,seq_rel_date_processed,download_url_base
0,GCA_000005825.2,PRJNA28811,SAMN02603086,NaN,representative genome,398511,79885,Bacillus pseudofirmus OF4,strain=OF4,NaN,...,15/12/2010,ASM582v2,"Center for Genomic Sciences, Allegheny-Singer ...",GCF_000005825.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2010-12-15,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
1,GCA_000005845.2,PRJNA225,SAMN02604091,NaN,reference genome,511145,562,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,NaN,...,26/09/2013,ASM584v2,Univ. Wisconsin,GCF_000005845.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2013-09-26,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
2,GCA_000006175.2,PRJNA20131,SAMN00000040,NaN,representative genome,456320,2188,Methanococcus voltae A3,strain=A3,NaN,...,03/06/2010,ASM617v2,US DOE Joint Genome Institute (JGI-PGF),GCF_000006175.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2010-06-03,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
3,GCA_000006605.1,PRJNA13967,SAMEA3283089,NaN,representative genome,306537,38289,Corynebacterium jeikeium K411,NaN,NaN,...,27/06/2005,ASM660v1,Bielefeld Univ,GCF_000006605.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,NaN,2005-06-27,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...
4,GCA_000019345.1,PRJNA19087,SAMN02604063,NaN,representative genome,505682,134821,Ureaplasma parvum serovar 3 str. ATCC 27815,strain=ATCC 27815,NaN,...,26/02/2008,ASM1934v1,TIGR,GCF_000019345.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...,NaN,assembly from type material,2008-02-26,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000...


In [338]:
ncbi_species_metadata.columns

Index(['assembly_accession', 'bioproject', 'biosample', 'wgs_master',
       'refseq_category', 'taxid', 'species_taxid', 'organism_name',
       'infraspecific_name', 'isolate', 'version_status', 'assembly_level',
       'release_type', 'genome_rep', 'seq_rel_date', 'asm_name', 'submitter',
       'gbrs_paired_asm', 'paired_asm_comp', 'ftp_path',
       'excluded_from_refseq', 'relation_to_type_material',
       'seq_rel_date_processed', 'download_url_base'],
      dtype='object')

## Keep traits from short-listed species only

In [12]:
species_traits = species_traits_all[
    species_traits_all['species_taxid'].isin(ncbi_species_metadata['species_taxid'].values)
].reset_index(drop=True)
len(species_traits)

2359

In [15]:
species_traits.columns

Index(['species_tax_id', 'species', 'genus', 'family', 'order', 'class',
       'phylum', 'superkingdom', 'gram_stain', 'metabolism', 'pathways',
       'carbon_substrates', 'sporulation', 'motility', 'range_tmp',
       'range_salinity', 'cell_shape', 'isolation_source', 'd1_lo', 'd1_up',
       'd2_lo', 'd2_up', 'doubling_h', 'genome_size', 'gc_content',
       'coding_genes', 'optimum_tmp', 'optimum_ph', 'growth_tmp',
       'rRNA16S_genes', 'tRNA_genes', 'gram_stain.count', 'metabolism.count',
       'pathways.count', 'carbon_substrates.count', 'sporulation.count',
       'motility.count', 'range_tmp.count', 'range_salinity.count',
       'cell_shape.count', 'isolation_source.count', 'gram_stain.prop',
       'metabolism.prop', 'pathways.prop', 'carbon_substrates.prop',
       'sporulation.prop', 'motility.prop', 'range_tmp.prop',
       'range_salinity.prop', 'cell_shape.prop', 'isolation_source.prop',
       'd1_lo.count', 'd1_up.count', 'd2_lo.count', 'd2_up.count',
       'doub

In [349]:
for c in species_traits.columns:
    v = species_traits[c].iloc[0]
    print(c, type(v), isinstance(v, float))

species_tax_id <class 'numpy.int64'> False
species <class 'str'> False
genus <class 'str'> False
family <class 'str'> False
order <class 'str'> False
class <class 'str'> False
phylum <class 'str'> False
superkingdom <class 'str'> False
gram_stain <class 'float'> True
metabolism <class 'str'> False
pathways <class 'float'> True
carbon_substrates <class 'float'> True
sporulation <class 'float'> True
motility <class 'float'> True
range_tmp <class 'float'> True
range_salinity <class 'float'> True
cell_shape <class 'float'> True
isolation_source <class 'float'> True
d1_lo <class 'numpy.float64'> True
d1_up <class 'numpy.float64'> True
d2_lo <class 'numpy.float64'> True
d2_up <class 'numpy.float64'> True
doubling_h <class 'numpy.float64'> True
genome_size <class 'numpy.float64'> True
gc_content <class 'numpy.float64'> True
coding_genes <class 'numpy.float64'> True
optimum_tmp <class 'numpy.float64'> True
optimum_ph <class 'numpy.float64'> True
growth_tmp <class 'numpy.float64'> True
rRNA16S_

In [18]:
assert np.all(species_traits['growth_tmp'].notnull())

## Load CDS

In [103]:
cds_path_fmt = os.path.join(os.getcwd(), 'data/condensed_traits/sequences/{taxid}/{taxid}_cds_from_genomic.fna.gz')

In [104]:
idx = 110
specie_taxid = species_traits.iloc[idx]['species_taxid']
species_traits.iloc[[idx]][['species_taxid', 'species', 'phylum', 'growth_tmp', 'genome_size', 'coding_genes']]

,species_taxid,species,phylum,growth_tmp,genome_size,coding_genes
110,375,Bradyrhizobium japonicum,Proteobacteria,27.0,9075243.179,8530.667


In [105]:
with gzip.open(cds_path_fmt.format(taxid=specie_taxid), mode='rt') as f:
    record_dict = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [106]:
r = sorted(record_dict.keys())[0]

In [107]:
cds = record_dict[r]

In [108]:
cds

SeqRecord(seq=Seq('ATGTTCGCTGATCGCCTCAAAGATTACAACCTTGCCCTAGCGACCGTGCTTCAG...TGA', SingleLetterAlphabet()), id='lcl|AP012206.1_cds_BAL05302.1_1', name='lcl|AP012206.1_cds_BAL05302.1_1', description='lcl|AP012206.1_cds_BAL05302.1_1 [locus_tag=BJ6T_00010] [protein=hypothetical protein] [protein_id=BAL05302.1] [location=123..428] [gbkey=CDS]', dbxrefs=[])

In [109]:
cds.translate()

SeqRecord(seq=Seq('MFADRLKDYNLALATVLQSVNSFELVGVGLVLDVPTDIASRRIFSNRTSASWSN...CS*', HasStopCodon(ExtendedIUPACProtein(), '*')), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

## Load RNA

In [112]:
rna_path_fmt = os.path.join(os.getcwd(), 'data/condensed_traits/sequences/{taxid}/{taxid}_rna_from_genomic.fna.gz')

In [113]:
with gzip.open(rna_path_fmt.format(taxid=specie_taxid), mode='rt') as f:
    rna_dict = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [118]:
rna_id = sorted(rna_dict.keys())[20]
rna = rna_dict[rna_id]

In [119]:
rna

SeqRecord(seq=Seq('GGCGGGGTAGCTCAGCTGGTTAGAGCACGGGAATCATAATCCTGGGGTCGGGGG...CCA', SingleLetterAlphabet()), id='lcl|AP012206.1_trna_23', name='lcl|AP012206.1_trna_23', description='lcl|AP012206.1_trna_23 [locus_tag=BJ6T_24130] [product=tRNA-Met] [location=2487544..2487620] [gbkey=tRNA]', dbxrefs=[])

## Putting it together

In [120]:
def load_specie_sequences(specie_taxid):
    dna_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_genomic.fna.gz',
    )
    cds_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_cds_from_genomic.fna.gz',
    )
    rna_path = os.path.join(
        os.getcwd(), 
        f'data/condensed_traits/sequences/{specie_taxid}/{specie_taxid}_rna_from_genomic.fna.gz',
    )
    
    with gzip.open(dna_path, mode='rt') as f:
        dna = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
    
    with gzip.open(cds_path, mode='rt') as f:
        cds = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
        
    with gzip.open(rna_path, mode='rt') as f:
        rna = SeqIO.to_dict(SeqIO.parse(f, "fasta"))
        
    return dna, cds, rna

## Parse location information

In [ ]:
dna_dict, cds_dict, rna_dict = load_specie_sequences(specie_taxid)

In [161]:
cds_dict['lcl|AP012206.1_cds_BAL05308.1_7']

SeqRecord(seq=Seq('ATGTCAGTGATCGATCTATCGCGCGCCGATCCGCGCACGTTGATCGGCAAGGGA...TAG', SingleLetterAlphabet()), id='lcl|AP012206.1_cds_BAL05308.1_7', name='lcl|AP012206.1_cds_BAL05308.1_7', description='lcl|AP012206.1_cds_BAL05308.1_7 [locus_tag=BJ6T_00070] [protein=ribonucleoside-diphosphate reductase beta subunit] [protein_id=BAL05308.1] [location=complement(4970..6043)] [gbkey=CDS]', dbxrefs=[])

In [ ]:
description = (
    "lcl|CP042806.1_cds_QEE29787.1_3650 [locus_tag=FTW19_18435] "
    "[protein=peptide chain release factor 2] [exception=ribosomal slippage] "
    "[protein_id=QEE29787.1] [location=complement(join(4615525..4616595,4616597..4616665))] [gbkey=CDS]"
)

In [165]:
gff_path_fmt = os.path.join(os.getcwd(), 'data/condensed_traits/sequences/{taxid}/{taxid}_genomic.gff.gz')

In [301]:
annotations = gffutils.create_db(
    gff_path_fmt.format(taxid=specie_taxid), 
    ':memory:', 
    merge_strategy='replace',
)

In [169]:
for f in annotations.featuretypes():
    print(f)

CDS
exon
gene
rRNA
region
tRNA
tmRNA


In [269]:
c = 0
cds_lengths = []
for i, feature in enumerate(annotations.features_of_type('CDS')):
    start, end = feature.start, feature.end
    assert start < end
    cds_lengths.append((end - start) + 1)
    c += 1
    
print(c)
np.mean(cds_lengths), np.min(cds_lengths), np.max(cds_lengths)

8829


(885.5548759768943, 120, 17028)

In [302]:
for i, feature in enumerate(annotations.features_of_type('CDS')):
    length = (feature.end - feature.start) + 1
    assert length % 3 == 0, 'nope'

In [268]:
def extract_non_coding_locations(dna_dict, annotations):
    regions = sorted(dna_dict.keys())
    full_length = sum([
        len(dna_dict[region].seq)
        for region in regions
    ])
    
    all_indices = set(range(1, full_length + 1))
    coding_indices = set()
    
    for feature_type in ['CDS', 'exon']:
        for i, feature in enumerate(annotations.features_of_type(feature_type)):
            start, end, strand = feature.start, feature.end, feature.strand
            
            if strand == '-':
                prev_start = start
                start = full_length - (end - 1)
                end = full_length - (prev_start - 1)
                assert start < end
            
            coding_indices |= set(range(start, end + 1))
            
    remaining_indices = sorted(all_indices - coding_indices)
    
    non_coding_segments = []
    start, end = None, None
    for i in remaining_indices:
        if start is None:
            start = i
            end = start
        elif end + 1 != i:
            if end != start:
                non_coding_segments.append([start, end])
            else:
                non_coding_segments.append([start])
                
            start = i
            end = start
        elif i == remaining_indices[-1]:
            if start == i:
                non_coding_segments.append([start])
            else:
                non_coding_segments.append([start, i])
        else:
            end = i
        
    return remaining_indices, full_length, non_coding_segments

In [259]:
remaining_indices, full_length, non_coding_segments = extract_non_coding_locations(dna_dict, annotations)

100 * len(remaining_indices) / full_length

33.33924163475749

In [260]:
len(non_coding_segments)

4479

In [283]:
lengths = []
for s in non_coding_segments:
    if len(s) == 1:
        lengths.append(1)
    else:
        assert len(s) == 2
        start, end = s
        assert start < end, f'{start}, {end}'
        lengths.append((end - start) + 1)

In [284]:
np.mean(lengths), np.min(lengths), np.max(lengths)

(685.3476222371065, 1, 19661)

In [265]:
non_coding_segments[0], non_coding_segments[1]

([1, 122], [429, 483])

## Percentage of non-coding DNA in species of our dataset

In [300]:
%%time
non_coding_percentages = []
for i, taxid in enumerate(species_traits['species_taxid'].values):
    if i == 0 or (i + 1) % 10 == 0:
        print(f'Specie {i+1} / {len(species_traits)}')
    
    dna_dict, _, _ = load_specie_sequences(specie_taxid)
    
    annotations = gffutils.create_db(
        gff_path_fmt.format(taxid=taxid), 
        ':memory:', 
        merge_strategy='replace',
    )
    
    remaining_indices, full_length, _ = extract_non_coding_locations(dna_dict, annotations)
    perc = 100 * len(remaining_indices) / full_length
    non_coding_percentages.append(perc)

Specie 1 / 2359


KeyboardInterrupt: 